# Yatir land surface parameterization

In [ ]:
from timutils.git_tools import print_cwd_git_version
print_cwd_git_version()

In [ ]:
import holoviews as hv
from holoviews import opts
import os
from cartopy import crs as ccrs

from geoviews_tools import yatir_landuse_to_xarray

In [ ]:
hv.extension('bokeh')

#allow multiple plots in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
landuse = yatir_landuse_to_xarray()

In [ ]:
import geoviews as gv
import geoviews.feature as gf
lat_dim = hv.Dimension('south_north', label='Latitude', unit='deg N')
lon_dim = hv.Dimension('west_east', label='Longitude', unit='deg E')

In [ ]:
# TODO
#  X use Quadmesh, not Image
#  - index by lat, lon (not array index)
#  - show surrounding map
#  - draw box for Yatir on d02 maps
#  - better colormap
def make_maps(landuse, domain):
    """make a list of maps, one per WRF run, for the specified domain"""
    wrf_runs = landuse[domain].coords['WRFrun'].values
    maps_list = [gv.Dataset(landuse[domain].sel(WRFrun=this_run), kdims=['PFT', 'lon', 'lat']).to(
        gv.QuadMesh, 
        ['lon', 'lat'], 
        'LANDUSEF', 
        label=this_run + ", " + domain).opts(colorbar=True, cmap='Blues') #, projection=ccrs.PlateCarree()) 
                 for this_run in wrf_runs
                ]
    # draw a box around domain 03
    corner_indices = ([0, 0, -1, -1], [0, -1, -1, -0])
    d03_corners = list(zip(landuse['d03']['lon'].values[corner_indices], 
                           landuse['d03']['lat'].values[corner_indices]))
    d03_box = gv.Polygons(d03_corners).opts(style={'line_color': '#d95f02', 'color': None}) 
    for this_run in range(len(wrf_runs)):
        maps_list[this_run] = maps_list[this_run] * d03_box
    return(maps_list)
d02_maps = make_maps(landuse, 'd02')
d03_maps = make_maps(landuse, 'd03')

In [ ]:
hv.Layout(d02_maps)
hv.Layout(d03_maps)